<a href="https://colab.research.google.com/github/c0lbalt/Deep-Learning-w-PyTorch/blob/main/Convolutional_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# Convert MNIST Image Files into Tensor
# of 4-Dimensions(# of images, Height, Width, Color Channel)

transform = transforms.ToTensor()

In [18]:
# Train Data
train_data = datasets.MNIST(root='CNN_data', train=True, download=True, transform=transform)

In [19]:
# Test Data
test_data = datasets.MNIST(root='CNN_data', train=False, download=True, transform=transform)

In [20]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: CNN_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [21]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: CNN_data
    Split: Test
    StandardTransform
Transform: ToTensor()